In [2]:
import math
import re
from random import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

### Dataset

The "simple-wiki" dataset comprises pairs of equivalent sentences sourced from Wikipedia. Each example in the dataset is structured as a dictionary with the key "set" and a list containing two sentences as the corresponding "value".

Supported Tasks:

Sentence Transformers Training: The dataset is specifically designed for training Sentence Transformers models. These models are valuable for tasks related to semantic search and sentence similarity.
Languages:

The dataset is available in English.

Source: https://huggingface.co/datasets/embedding-data/simple-wiki

The code reads data from a Parquet file located at the specified path using Pandas. The resulting data is stored in a DataFrame, enabling further exploration, analysis, and manipulation of the data using Pandas functionalities. Ensure that the necessary libraries are installed (pandas and pyarrow) before running this code.

In [3]:
import pandas as pd
parquet_file = "data/0000.parquet"
df = pd.read_parquet(parquet_file, engine='pyarrow')

In [4]:
text = ''
for idx,rows in df.iterrows():
    if idx > 20:
        break
    for sent in rows['set']:
        # print(sent)
        text += sent+'\n'

The code aggregates sentences from the 'set' column of the first 21 rows of the DataFrame df into a single string (text). The loop breaks after processing 21 rows. This type of operation might be useful for creating a consolidated text summary or for further text processing.

In [5]:
import spacy

nlp = spacy.load("en_core_web_sm")
# nlp.max_length = 3030000 # or higher
doc = nlp(text)
sentences = list(doc.sents)

#lower case, and clean all the symbols
text = [x.text.lower() for x in sentences]
text = [re.sub("[.,!?\\-]", '', x) for x in text]

Tokenizes the text into sentences, converts them to lowercase, and removes specified punctuation symbols. 

In [6]:
#making vocabs - numericalization
word_list = list(set(" ".join(text).split()))
word2id   = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}

Creates a vocabulary and corresponding word-to-id mapping for a list of cleaned sentences stored in the variable text. The vocabulary includes unique words in the text, and some special tokens like '[PAD]', '[CLS]', '[SEP]', and '[MASK]' are assigned specific identifiers in the word-to-id mapping. 

In [7]:
for i, w in enumerate(word_list):
    word2id[w] = i + 4 #reserve the first 0-3 for CLS, PAD
    id2word    = {i:w for i, w  in enumerate(word2id)}
    vocab_size = len(word2id)
    
token_list = list()
for sentence in sentences:
    arr = [word2id[word] for sentence in text for word in sentence.split()]
    token_list.append(arr)

Extends the vocabulary and word-to-id mapping created earlier and then uses this mapping to convert sentences into tokenized representations.

In [8]:
batch_size = 6
max_mask   = 5 #even though it does not reach 15% yet....maybe you can set this threshold
max_len    = 2000 #maximum length that my transformer will accept.....all sentence will be padded

In [9]:
def make_batch():
    batch = []
    positive = negative = 0
    while positive != batch_size / 2 or negative != batch_size / 2:
        
        #randomly choose two sentence
        tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange(len(sentences))
        tokens_a, tokens_b            = token_list[tokens_a_index], token_list[tokens_b_index]
        
        #1. token embedding - add CLS and SEP
        input_ids = [word2id['[CLS]']] + tokens_a + [word2id['[SEP]']] + tokens_b + [word2id['[SEP]']]
        
        #2. segment embedding - which sentence is 0 and 1
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)
        
        #3 masking
        n_pred = min(max_mask, max(1, int(round(len(input_ids) * 0.15))))
        #get all the pos excluding CLS and SEP
        candidates_masked_pos = [i for i, token in enumerate(input_ids) if token != word2id['[CLS]'] 
                                 and token != word2id['[SEP]']]
        shuffle(candidates_masked_pos)
        masked_tokens, masked_pos = [], []
        #simply loop and mask accordingly
        for pos in candidates_masked_pos[:n_pred]:
            masked_pos.append(pos)
            masked_tokens.append(input_ids[pos])
            if random() < 0.1:  #10% replace with random token
                index = randint(0, vocab_size - 1)
                input_ids[pos] = word2id[id2word[index]]
            elif random() < 0.8:  #80 replace with [MASK]
                input_ids[pos] = word2id['[MASK]']
            else: 
                pass
            
        #4. pad the sentence to the max length
        n_pad = max_len - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)
        
        #5. pad the mask tokens to the max length
        if max_mask > n_pred:
            n_pad = max_mask - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)
        
        #6. check whether is positive or negative
        if tokens_a_index + 1 == tokens_b_index and positive < batch_size / 2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True])
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size / 2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False])
            negative += 1
        
    return batch

Generates batches of training data that can predict masked tokens within a sentence.

In [10]:
batch = make_batch()

In [11]:
len(batch)

6

In [12]:
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))


In [13]:
input_ids.shape, segment_ids.shape, masked_tokens.shape, masked_pos.shape, isNext

(torch.Size([6, 2000]),
 torch.Size([6, 2000]),
 torch.Size([6, 5]),
 torch.Size([6, 5]),
 tensor([0, 0, 0, 1, 1, 1]))

In [14]:
masked_tokens

tensor([[313, 221,  35, 278,  56],
        [138, 234, 143, 253,   5],
        [148, 155,  73, 154, 140],
        [168, 217, 240, 251, 323],
        [261, 141,  85,  47,  19],
        [168,  50, 163, 234, 110]])

In [15]:
n_layers = 6    # number of Encoder of Encoder Layer
n_heads  = 8    # number of heads in Multi-Head Attention
d_model  = 768  # Embedding Size
d_ff = 768 * 4  # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_segments = 2

In [16]:
class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
        self.pos_embed = nn.Embedding(max_len, d_model)      # position embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):
        #x, seg: (bs, len)
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long)
        pos = pos.unsqueeze(0).expand_as(x)  # (len,) -> (bs, len)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

Embedding class, which is designed to handle token, position, and segment embeddings.

In [17]:
def get_attn_pad_mask(seq_q, seq_k):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

**get_attn_pad_mask** generates an attention mask to handle padding in a sequence. This mask is intended for use in Bert model to prevent attention from being applied to padding tokens. 

In [18]:
print(get_attn_pad_mask(input_ids, input_ids).shape)


torch.Size([6, 2000, 2000])


In [19]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn 

**ScaledDotProductAttention** represents the scaled dot-product attention mechanism commonly used in Bert.

In [20]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]

        # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) # context: [batch_size x len_q x n_heads * d_v]
        output = nn.Linear(n_heads * d_v, d_model)(context)
        return nn.LayerNorm(d_model)(output + residual), attn # output: [batch_size x len_q x d_model]

**MultiHeadAttention** represents the multi-head attention mechanism in a Bert model. 

In [21]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # (batch_size, len_seq, d_model) -> (batch_size, len_seq, d_ff) -> (batch_size, len_seq, d_model)
        return self.fc2(F.gelu(self.fc1(x)))

**PoswiseFeedForwardNet** represents the position-wise feedforward network within a Bert model. This network consists of two linear layers with a GELU activation function.

In [22]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn       = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs, attn

**EncoderLayer**  represents one layer of the encoder in a Bert model. This layer consists of a multi-head self-attention mechanism (MultiHeadAttention) followed by a position-wise feedforward network (PoswiseFeedForwardNet). 

In [102]:
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.embedding = Embedding()
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        # decoder is shared with embedding layer
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_mode, d_model]
        
        # 1. predict next sentence
        # it will be decided by first token(CLS)
        h_pooled   = self.activ(self.fc(output[:, 0])) # [batch_size, d_model]
        logits_nsp = self.classifier(h_pooled) # [batch_size, 2]

        # 2. predict the masked token
        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) # [batch_size, max_pred, d_model]
        h_masked = torch.gather(output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
        h_masked  = self.norm(F.gelu(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias # [batch_size, max_pred, n_vocab]

        return logits_lm, logits_nsp
    

##### Pytorch imlementation of Bert

Embedding Layer:

The model starts with an embedding layer (Embedding), responsible for token, position, and segment embeddings.
Encoder Layers:

The model consists of multiple encoder layers (EncoderLayer) organized as a stack (nn.ModuleList). Each encoder layer includes a multi-head self-attention mechanism and a position-wise feedforward network, which are crucial components of the Transformer architecture.
Additional Linear and Activation Layers:

After the encoder layers, the model includes additional linear and activation layers for downstream tasks:
fc: A linear layer followed by a hyperbolic tangent (Tanh) activation function.
linear: Another linear layer.
norm: A layer normalization operation.
classifier: A linear layer for predicting the next sentence in a binary classification task.
Decoder Layer for Masked Language Model (MLM):

The decoder for the masked language model (MLM) task is shared with the embedding layer.
The weights of the decoder are tied to the token embedding weights.
A linear layer (decoder) followed by a bias term is used to predict the masked tokens in the input sequence.
Forward Method:

The forward method takes input_ids (token indices), segment_ids (segment indices), and masked_pos (positions of masked tokens) as inputs.
The input undergoes embedding, followed by multiple encoder layers.
The encoder self-attention mask is generated to handle padding tokens.
The output from the encoder layers is used for two tasks:
Predicting the next sentence (logits_nsp) based on the first token's representation.
Predicting masked tokens (logits_lm) by attending to the masked positions.

In [24]:
num_epoch = 500
model = BERT()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

batch = make_batch()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

for epoch in range(num_epoch):
    optimizer.zero_grad()
    logits_lm, logits_nsp = model(input_ids, segment_ids, masked_pos)    
    #logits_lm: (bs, max_mask, vocab_size) ==> (6, 5, 34)
    #logits_nsp: (bs, yes/no) ==> (6, 2)

    #1. mlm loss
    #logits_lm.transpose: (bs, vocab_size, max_mask) vs. masked_tokens: (bs, max_mask)
    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens) # for masked LM
    loss_lm = (loss_lm.float()).mean()
    #2. nsp loss
    #logits_nsp: (bs, 2) vs. isNext: (bs, )
    loss_nsp = criterion(logits_nsp, isNext) # for sentence classification
    
    #3. combine loss
    loss = loss_lm + loss_nsp
    if epoch % 100 == 0:
        print('Epoch:', '%02d' % (epoch), 'loss =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

c:\Users\PREDATOR\Music\ML_project\nlp\nlp-assignment-a3\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch: 00 loss = 84.624443
Epoch: 100 loss = 4.239068
Epoch: 200 loss = 3.811649
Epoch: 300 loss = 3.803038
Epoch: 400 loss = 3.716961


End-to-end training process for a BERT model on a above dataset. The model is simultaneously trained for both masked language modeling and next sentence prediction tasks, with the overall loss being a combination of both. The synthetic dataset is generated using the make_batch() function, and the training loop updates the model parameters to minimize the combined loss over the specified number of epochs.

In [155]:
#Saving the model
torch.save(model.state_dict(), 'bert_model.pth')

In [156]:
#Loading the model
pretrained_model = model.load_state_dict(torch.load('bert_model.pth'))

In [147]:
# Predict mask tokens ans isNext
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[2]))
print([id2word[w.item()] for w in input_ids[0] if id2word[w.item()] != '[PAD]'])

logits_lm, logits_nsp = pretrained_model(input_ids, segment_ids, masked_pos)
#logits_lm:  (1, max_mask, vocab_size) ==> (1, 5, 34)
#logits_nsp: (1, yes/no) ==> (1, 2)

#predict masked tokens
#max the probability along the vocab dim (2), [1] is the indices of the max, and [0] is the first value
logits_lm = logits_lm.data.max(2)[1][0].data.numpy() 
#note that zero is padding we add to the masked_tokens
print('masked tokens (words) : ',[id2word[pos.item()] for pos in masked_tokens[0]])
print('masked tokens list : ',[pos.item() for pos in masked_tokens[0]])
print('masked tokens (words) : ',[id2word[pos.item()] for pos in logits_lm])
print('predict masked tokens list : ', [pos for pos in logits_lm])

#predict nsp
logits_nsp = logits_nsp.data.max(1)[1][0].data.numpy()
print(logits_nsp)
print('isNext : ', True if isNext else False)
print('predict isNext : ',True if logits_nsp else False)

['[CLS]', 'the', 'greatest', 'example', 'has', 'been', 'in', 'his', 'present', 'job', '(', 'then', 'minister', 'for', 'foreign', 'affairs', ')', 'where', 'he', 'has', 'perforce', 'concentrated', 'on', 'angloirish', 'relations', 'and', 'in', 'particular', 'northern', 'ireland', '(', 'the', 'greatest', 'example', 'has', 'been', 'in', 'his', 'present', 'job', '[MASK]', 'then', 'minister', 'for', 'foreign', 'affairs', ')', 'where', 'he', 'has', 'perforce', 'concentrated', 'on', 'angloirish', 'relations', 'and', 'in', 'particular', 'the', 'north', '(', 'ie', 'northern', 'ireland', ')', 'president', 'hillery', 'refused', 'to', 'speak', 'to', 'any', 'opposition', 'party', 'politicians', 'but', 'when', 'charles', 'haughey', 'who', 'was', 'leader', 'of', 'the', 'opposition', 'had', 'rang', 'the', 'president', "'s", 'office', 'he', 'threatened', 'to', 'end', 'the', 'career', 'of', 'the', 'army', 'officer', 'answered', 'and', 'refused', 'on', 'hillery', "'s", 'explicit', 'orders', 'to', 'put', 't

TypeError: '_IncompatibleKeys' object is not callable

In [111]:
input_ids

tensor([[  1, 234, 175,  ...,   0,   0,   0]])

##### Task 2. Sentence Embedding with Sentence BERT

In [130]:
from transformers import BertTokenizer
pretrained_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [131]:
from datasets import load_dataset

# Load the SNLI dataset
dataset = load_dataset('snli', split='test')

In [132]:
def encode_examples(examples):
    # Tokenize the sentence pairs
    # tokenized_data = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    return pretrained_tokenizer(examples['premise'], return_tensors='pt', padding=True, truncation=True)
    # return pretrained_model(examples['premise'], examples['hypothesis'], padding='max_length', truncation=True, return_tensors='pt')

In [133]:
encoded_dataset = dataset.map(encode_examples, batched=True)

In [47]:
# Load the tokenizer and BERT model
# tokenizer = model.from_pretrained('bert_model')
# model = BertModel.from_pretrained('bert-base-uncased')


In [115]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [122]:
def encode_examples(examples):
    # Tokenize the sentence pairs
    # tokenized_data = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    return pretrained_tokenizer(examples['premise'], return_tensors='pt', padding=True, truncation=True)
    # return pretrained_model(examples['premise'], examples['hypothesis'], padding='max_length', truncation=True, return_tensors='pt')

In [123]:
# Encode the SNLI dataset
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenized_data = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

encoded_dataset = dataset.map(encode_examples, batched=True)

Map: 100%|██████████| 10000/10000 [00:04<00:00, 2333.11 examples/s]


In [118]:
from torch.utils.data import DataLoader

class SiameseNetwork(nn.Module):
    def __init__(self, bert_model, fc_dim=768):
        super(SiameseNetwork, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(fc_dim, 1)

    def forward(self, premise_input_ids, premise_token_type_ids, premise_attention_mask,
                hypothesis_input_ids, hypothesis_token_type_ids, hypothesis_attention_mask):
        # Forward pass for premise
        _, premise_embedding = self.bert(
            input_ids=premise_input_ids,
            token_type_ids=premise_token_type_ids,
            attention_mask=premise_attention_mask
        )

        # Forward pass for hypothesis
        _, hypothesis_embedding = self.bert(
            input_ids=hypothesis_input_ids,
            token_type_ids=hypothesis_token_type_ids,
            attention_mask=hypothesis_attention_mask
        )

        # Compute cosine similarity
        cosine_similarity = self.compute_cosine_similarity(premise_embedding, hypothesis_embedding)

        # Apply fully connected layer
        output = self.fc(cosine_similarity)

        return output

    def compute_cosine_similarity(self, x1, x2):
        # Compute cosine similarity between two vectors
        dot_product = torch.sum(x1 * x2, dim=-1)
        norm_x1 = torch.norm(x1, dim=-1)
        norm_x2 = torch.norm(x2, dim=-1)
        cosine_similarity = dot_product / (norm_x1 * norm_x2 + 1e-8)  # Add epsilon to avoid division by zero
        return cosine_similarity

    def classification_loss(self, logits, label):
        # Classification objective function with softmax
        softmax = nn.Softmax(dim=1)
        probabilities = softmax(logits)
        loss = nn.CrossEntropyLoss()(probabilities, label)
        return loss

In [157]:
dataloader = DataLoader(encoded_dataset, batch_size=batch_size)
siamese_model = SiameseNetwork(pretrained_model, fc_dim=768)
batch_size = 6
dataloader = DataLoader(encoded_dataset, batch_size=batch_size)
# Set up the optimizer
num_epoch = 500
embedding_dim = 768 
# Assuming you have data with keys ['premise', 'hypothesis', 'label']
# You need to tokenize the input sentences and prepare input tensors
premise_text = "This is a sample premise."
hypothesis_text = "This is a sample hypothesis."
label = torch.tensor([1])  # Example label, can be 0 or 1

# Tokenize input sentences
premise_tokens = pretrained_tokenizer(premise_text, return_tensors='pt', padding=True, truncation=True)
hypothesis_tokens = pretrained_tokenizer(hypothesis_text, return_tensors='pt', padding=True, truncation=True)

# Extract input tensors
premise_input_ids = premise_tokens['input_ids']
premise_token_type_ids = premise_tokens['token_type_ids']
premise_attention_mask = premise_tokens['attention_mask']

hypothesis_input_ids = hypothesis_tokens['input_ids']
hypothesis_token_type_ids = hypothesis_tokens['token_type_ids']
hypothesis_attention_mask = hypothesis_tokens['attention_mask']

# Forward pass through the Siamese network
output = siamese_model(
    premise_input_ids=premise_input_ids,
    premise_token_type_ids=premise_token_type_ids,
    premise_attention_mask=premise_attention_mask,
    hypothesis_input_ids=hypothesis_input_ids,
    hypothesis_token_type_ids=hypothesis_token_type_ids,
    hypothesis_attention_mask=hypothesis_attention_mask
)

# Calculate classification loss
loss = siamese_model.classification_loss(output, label)

print("Siamese Network Output:", output.item())
print("Classification Loss:", loss.item())